# 加载必要的工具库

In [1]:
import numpy as np
import os

# 加载数据集

In [2]:
def load_data(file: str, sep: str = ' ') -> tuple:
    """
    从文件中加载数据。要求数据文件每行包含一个样例且每行最后一个数据为样例标签。
    input:
        file: 数据文件路径。
        sep: 文件中每个特征的分隔符。
    output:
        X: 每行表示一个样本、每列表示一个样本特征的矩阵。
        Y: 每行表示一个样本标签、列数为 1 的向量。
    """
    with open(file=file, mode='r') as f:
        data = np.array([[
            np.float64(feature)
            for feature in sample.strip().split(sep=sep)]
            for sample in f.readlines()])

    X = data[:, :-1]
    Y = data[:, -1]

    return X, Y

## 加载西瓜数据集

In [3]:
X_training, Y_training = load_data('dataset/melon_training.txt', ',')
X_test, Y_test = load_data('dataset/melon_test.txt', ',')

## 加载鸢尾花数据集
由于鸢尾花有三个分类标签，分类的思路是：训练三个 logistic regression 模型，每个模型分别计算样本属于三个分类的概率，取三者中概率最大者的分类为样本的分类。

加载数据时每个训练集、测试集把一个分类标签视作 1，其他两个分类标签视作 0。

In [4]:
iris_X_training, iris_Y_training = load_data(
    file='dataset/iris_training.txt', sep=',')
iris_Y_training_0 = np.array(
    [np.float64(1.) if y == 0. else np.float64(0.) for y in iris_Y_training])
iris_Y_training_1 = np.array(
    [np.float64(1.) if y == 1. else np.float64(0.) for y in iris_Y_training])
iris_Y_training_2 = np.array(
    [np.float64(1.) if y == 2. else np.float64(0.) for y in iris_Y_training])

iris_X_test, iris_Y_test = load_data(file='dataset/iris_test.txt', sep=',')
iris_Y_test_0 = np.array(
    [np.float64(1.) if y == 0. else np.float64(0.) for y in iris_Y_test])
iris_Y_test_1 = np.array(
    [np.float64(1.) if y == 1. else np.float64(0.) for y in iris_Y_test])
iris_Y_test_2 = np.array(
    [np.float64(1.) if y == 2. else np.float64(0.) for y in iris_Y_test])

# 加载参数
西瓜数据集的参数保存在 `./parameters.txt` 中。
鸢尾花数据集的参数保存在 `./parameters_0.txt`、`./parameters_1.txt`、`./parameters_2.txt` 中。

In [5]:
def load_parameters(file: str) -> tuple:
    """
    加载文件中的参数。
    input:
        file: 保留参数的文件路径。
    output:
        文件中保留的 weights 和 bias。
    """
    with open(file=file, mode='r') as f:
        para = f.readline().strip().split(',')
        weights = np.array(para[:-1], dtype=np.float64)
        bias = np.array(para[-1:], dtype=np.float64)
    return weights, bias

## 西瓜数据集的参数

In [6]:
if not os.path.exists('parameters.txt'):
    melon_weights, melon_bias = np.random.rand(2), np.random.rand(1)
else:
    melon_weights, melon_bias = load_parameters('parameters.txt')

## 鸢尾花数据集的参数
与鸢尾花的三个模型对应有三组参数。

In [7]:
if not os.path.exists('parameters_0.txt'):
    weights_0, bias_0 = np.random.rand(4), np.random.rand(1)
else:
    weights_0, bias_0 = load_parameters('parameters_0.txt')
if not os.path.exists('parameters_1.txt'):
    weights_1, bias_1 = np.random.rand(4), np.random.rand(1)
else:
    weights_1, bias_1 = load_parameters('parameters_1.txt')
if not os.path.exists('parameters_2.txt'):
    weights_2, bias_2 = np.random.rand(4), np.random.rand(1)
else:
    weights_2, bias_2 = load_parameters('parameters_2.txt')

# Sigmoid 函数
$$\operatorname{Sigmoid}\left(z\right) = \frac{1}{1 + \exp\left(-z\right)}$$

In [8]:
def sigmoid(z: np.array) -> np.array:
    """
    Sigmoid(z) = 1 / (1 + exp(-z))。
    """
    return np.float64(1.) / (np.float64(1.) + np.exp(-z))

# 预测分类
$$P\left(y=1\right) = \operatorname{Sigmoid}\left(\vec{w}^T\cdot\vec{x}+b\right)$$
$$P\left(y=0\right) = 1 - P\left(y=1\right)$$

In [9]:
def predict(X: np.matrix, weights: np.array, bias: np.array) -> np.array:
    """
    根据样本和权重预测样本的标签。
    P{y = 1 | X} = sigmoid(w^T x + b)。
    input:
        X: 同 load_data 中的 X。
        weights: 行数为样本特征数、列数为 1 的权重向量。即公式中的 w。
        bias: 即公式中的 b。
    output:
        与 load_data 中 Y 形状相同、对 Y 的预测值。
    """
    positive = sigmoid(np.dot(X, weights) + bias)
    negative = 1 - positive
    return np.argmax(np.array([negative, positive]), axis=0)


def multi_predict(X: np.matrix, weights: np.matrix, bias: np.array) -> np.array:
    """
    根据样本和权重预测样本的标签。
    是 predict 针对多分类问题的改进。
    假设共有 C 个类别、训练 C 个 logistic regression 模型、
    预测时计算样本属于每个类的概率、取概率最大者作为样本的概率。
    input:
        X: 同 predict 中的 X。
        weights: 与 predict 中的不同、此处 weights 有 C 行、每行对应 predict 中
            一个 logistic regression 模型的 weights。
        bias: 与 predict 中的不同。此处 bias 为 C 行 1 列的向量。
    output:
        与 load_data 中 Y 形状相同、对 Y 的预测值。
    """
    pred = sigmoid(np.matmul(X, weights.T) +
                   np.matmul(np.ones((X.shape[0], 1), dtype=np.float64), bias.T))
    return np.argmax(pred, axis=1)

# 损失函数
单个训练样本的损失函数为$$L\left(f\left(x^{\left(i\right)}\right),y^{\left(i\right)}\right)=\left\{\begin{align}-\mathop{ln}\left(f\left(\vec{x}^{\left(i\right)}\right)\right),y^{\left(i\right)}=1,\\-\mathop{ln}\left(1-f\left(\vec{x}^{\left(i\right)}\right)\right),y^{\left(1\right)}=0\end{align}\right.\\=-y^{\left(i\right)}\mathop{ln}\left(f\left(x^{\left(i\right)}\right)\right)-\left(1-y^{\left(i\right)}\right)\mathop{ln}\left(1-f\left(x^{\left(i\right)}\right)\right)$$
数据集整体的损失函数为
$$J\left(\vec{w},b\right)=\frac{1}{m}\sum\limits_{i=1}^{m}{L\left(f\left(\vec{x}^{\left(i\right)}\right),y^{\left(i\right)}\right)}$$
# 梯度下降法优化损失函数
$$w_j:=w_j-\alpha\frac{\partial}{\partial{w_j}}J\left(\vec{w},b\right)$$
$$b:=b-\alpha\frac{\partial}{\partial{b}}J\left(\vec{w},b\right)$$
其中
$$\frac{\partial}{\partial{w_j}}J\left(\vec{w},b\right)=\frac{1}{m}\sum\limits_{i=1}^{m}{\left(f\left(\vec{x}^{\left(i\right)}\right)-y^{\left(i\right)}\right)x_j^{\left(i\right)}}$$
$$\frac{\partial}{\partial{b}}J\left(\vec{w},b\right)=\frac{1}{m}\sum\limits_{i=1}^{m}{\left(f\left(\vec{x}^{\left(i\right)}\right)-y^{\left(i\right)}\right)}$$

In [10]:
def gradient_descent(X: np.matrix, Y: np.array, weights: np.array, bias: np.array, lr: np.float64) -> None:
    """
    采用梯度下降法优化损失函数。损失函数采用交叉熵损失、其公式为
        loss = -y ln y - (1 - y) ln(1 - y)。
    对应的参数更新公式为
        w_j := w_j - alpha / m * sum_i ((yhat_i - y_i) * x_i_j)
        b := b - alpha / m * sum_i (yhat_i - y_i)
    input:
        X: 同 load_data 中的 X。
        Y: 同 load_data 中的 Y。
        weights: 同 predict 中的 weights。
        bias: 同 predict 中的 bias。
        lr: 学习率。即公式中的 alpha
    """
    predictions = sigmoid(np.dot(X, weights) + bias)
    weights -= lr * np.dot(predictions - Y, X) / len(Y)
    bias -= lr * np.dot(np.ones((1, len(Y)), dtype=np.float64),
                        predictions - Y) / len(Y)

# 训练模型参数

## 训练西瓜数据集参数

In [11]:
lr = 0.05
epoch = 10000
for _ in range(epoch):
    gradient_descent(X=X_training, Y=Y_training, weights=melon_weights, bias=melon_bias, lr=lr)

## 训练鸢尾花数据集参数

In [12]:
lr = 0.05
epoch = 10000
for _ in range(epoch):
    gradient_descent(X=iris_X_training, Y=iris_Y_training_0,
                     weights=weights_0, bias=bias_0, lr=lr)
    gradient_descent(X=iris_X_training, Y=iris_Y_training_1,
                     weights=weights_1, bias=bias_1, lr=lr)
    gradient_descent(X=iris_X_training, Y=iris_Y_training_2,
                     weights=weights_2, bias=bias_2, lr=lr)

# 对模型进行测试

In [13]:
def test(X: np.matrix, Y: np.array, weights: np.array, bias: np.array) -> None:
    """
    根据给定的测试集和模型参数、测试模型的正确率。
    input:
        X: 同 load_data 中的 X。
        Y: 同 load_data 中的 Y。
        weights: 同 predict 中的 weights。
        bias: 同 predict 中的 bias。
    """
    predictions = predict(X=X, weights=weights, bias=bias)
    right = 0
    error = 0
    for i in range(len(Y)):
        if predictions[i] == Y[i]:
            right += 1
        else:
            error += 1
    print("Right: {}, error: {}, right rate: {}".format(
        right, error, right / (right + error)))


def multi_test(X: np.matrix, Y: np.array, weights: np.array, bias: np.array) -> None:
    """
    根据给定的测试集和模型参数、测试模型的正确率。
    是 test 针对多分类问题的改进。
    input:
        X: 同 load_data 中的 X。
        Y: 同 load_data 中的 Y。
        weights: 同 multi_predict 中的 weights。
        bias: 同 multi_predict 中的 bias。
    """
    predictions = multi_predict(X=X, weights=weights, bias=bias)
    right = 0
    error = 0
    for i in range(len(Y)):
        if predictions[i] == Y[i]:
            right += 1
        else:
            error += 1
    print("Right: {}, error: {}, right rate: {}".format(
        right, error, right / (right + error)))

## 西瓜数据集测试

In [14]:
print('---- My Logistic Regression ----\nWeights: \n{}\nBias: \n{}'.format(melon_weights, melon_bias))
test(X=X_test, Y=Y_test, weights=melon_weights, bias=melon_bias)

---- My Logistic Regression ----
Weights: 
[ 3.05101701 12.10187337]
Bias: 
[-4.28504056]
Right: 12, error: 5, right rate: 0.7058823529411765


## 鸢尾花数据集测试

In [15]:
weights = np.matrix([weights_0, weights_1, weights_2])
bias = np.matrix([bias_0, bias_1, bias_2])
print('---- My Multi-Class Logistic Regression ----\nWeights: \n{}\nBias: \n{}'.format(weights, bias))
print('In training set:')
multi_test(X=iris_X_training, Y=iris_Y_training, weights=weights, bias=bias)
print('In test set:')
multi_test(X=iris_X_test, Y=iris_Y_test, weights=weights, bias=bias)

---- My Multi-Class Logistic Regression ----
Weights: 
[[ 1.0325552   3.07961647 -5.25003492 -2.15822665]
 [ 0.06411905 -2.96329357  0.99255406 -2.48007079]
 [-3.76936662 -4.93324282  6.19725336  9.36306581]]
Bias: 
[[ 0.74913973]
 [ 6.75277965]
 [-8.46841086]]
In training set:
Right: 103, error: 2, right rate: 0.9809523809523809
In test set:
Right: 44, error: 1, right rate: 0.9777777777777777


# 保存参数

In [16]:
def save_parameters(file: str, weights: np.array, bias: np.array) -> None:
    """
    将参数保留到文件中。
    input:
        file: 保留参数的文件路径。
        weights: 同 predict 中的 weights。
        bias: 同 predict 中的 bias。
    """
    with open(file=file, mode='w') as f:
        for w in weights:
            f.write('{},'.format(w))
        f.write('{}'.format(bias[0]))

In [17]:
save_parameters('parameters.txt', weights=melon_weights, bias=melon_bias)

save_parameters('parameters_0.txt', weights=weights_0, bias=bias_0)
save_parameters('parameters_1.txt', weights=weights_1, bias=bias_1)
save_parameters('parameters_2.txt', weights=weights_2, bias=bias_2)